In [ ]:
import pandas as pd
import csv
import datetime
from dateutil.parser import parse

In [ ]:
df = pd.read_excel(r'RAW FILE.xlsx')

In [ ]:
if "Visit Count" in df.columns:
    df.drop('Visit Count', axis = 1,inplace = True)

In [ ]:
row_count = sum(1 for row in df["Provider"])

In [ ]:
if df["Provider"][row_count-1].strip() == "Grand Total":
    df.drop(df.tail(1).index, inplace=True)
    row_count-=1

In [ ]:
#deleting rows where names are included in Visit Id or Visit Day column or if visit id is in visit day column

i = row_count-1
x = i-1
while i > 0:
    visitId = df["Visit ID"][i]
    visitDay = df["Visit Day"][i]
    provider = df["Provider"][i]
    clinic_name = str(df["Visit Clinic Name"][i])
    
    if "(" in str(visitId) or "(" in str(visitDay) or "/" in str(visitId) or "/" not in str(visitDay) or "wellnow" not in str(clinic_name.lower()) or\
    "subtotal" in str(visitId).lower() or "subtotal" in str(visitDay).lower() or "subtotal" in str(clinic_name).lower() or "subtotal" in str(provider).lower():
        df.drop(df.index[i], inplace=True)
        row_count -= 1
        i = x
        x = i-1
    else:
        i-=1
        x = i-1

visitid0 = str(df["Visit ID"][0]).split()[-1]
visitday0 = str(df["Visit Day"][0]).split()[-1]
clinic0 = str(df["Visit Clinic Name"][0]).lower()
provider0 = df["Provider"][i].lower()

if "(" in visitid0 or "(" in visitday0 or "/" in visitid0 or "/" not in visitday0 or "wellnow" not in clinic0.lower() or\
    "subtotal" in visitid0.lower() or "subtotal" in visitday0 or "subtotal" in clinic0.lower() or "subtotal" in provider0.lower():
        df.drop(df.index[0], inplace=True)
        row_count -= 1

In [ ]:
print(df.head(10))

In [ ]:
dftop = df.head()
l1 = []
for row in df.index: 
    l1.append(row)
# print(l1)

In [ ]:
provider = []

for i in l1:
    val = str(df["Provider"][i]).split()
    del val[-1]
    val = val[::-1]
    
    if len(val) == 2:
        v = val[0]+','+' '+val[1]
    elif len(val) == 1:
        v = val[0]
    else:
        v = val
    provider.append(v)

In [ ]:
vid = []

for i in l1:
    v_id = int(df["Visit ID"][i])
    vid.append(v_id)

In [ ]:
dates = []

dos = []
for i in l1:
    d2 = datetime.datetime.strptime(df["Visit Day"][i],'%m/%d/%Y').strftime('%m/%d/%y')
    dos.append(d2)
    
    d3 = datetime.datetime.strptime(d2,'%m/%d/%y')
    df["Visit Day"][i] = d3
    dates.append(d3)

In [ ]:
visit_clinic_name = []
for i in l1:
    visit_clinic_val = df["Visit Clinic Name"][i]
    visit_clinic_name.append(visit_clinic_val)

# copy the data into the TEMPLATE file

In [ ]:
df["Provider"] = provider
df["Visit ID"] = vid
df["Visit Day"] = dates
df["Visit Clinic Name"] = visit_clinic_name

writer = pd.ExcelWriter(r"OUTPUT FILE.xlsx", engine = 'xlsxwriter')
df0 = pd.DataFrame(list(zip(provider,vid,dates,dos,visit_clinic_name)), columns =['Provider','Visit ID', 'Visit Day','DOS','Visit Clinic Name'])
df0.to_excel(writer, sheet_name = "Table1", index = False)
# df.to_excel(r"C:\Users\w501738\Desktop\chartsnotlockedTEST.xlsx",index = False)

In [ ]:
writer.save()
writer.close()